## Indicate Dynawo and Modelica package.mo

In [1]:
#=  The env variable DYNAWO_PATH must has be set beforehands to the root of the dynawo code.
    In case you are only working with the library distribution, the second part of the expression must be slightly modified.
    Anyway the package_file should be pointing to the package.mo file of the Dynawo library you want to use
=#

Dynawo_package_file = string(ENV["DYNAWO_PATH"], "/dynawo/sources/Models/Modelica/Dynawo/package.mo")
Modelica_package_file = string(ENV["MODELICA_PATH"], "package.mo")

"/home/guironnetadr/Projects/Install/OpenModelica/lib/omlibrary/Modelica/package.mo"

## Import packages and data

In [2]:
using OMJulia
using Plots, DataFrames, CSV, LinearAlgebra

In [3]:
Kundur = OMJulia.OMCSession()
ModelicaSystem(Kundur,Dynawo_package_file,"Dynawo.Examples.TwoAreas.KundurPSS",[Modelica_package_file])


[ Info: Path to zmq file="/tmp/openmodelica.guironnetadr.port.julia.jqW2IHOIsm"


## Run initial simulation

In [4]:
buildModel(Kundur)
setLinearizationOptions(Kundur,["stopTime=20","tolerance=1e-04"])
resultfile_name = string("Kundur", ".csv")
simulate(Kundur,resultfile = resultfile_name,
                simflags = "-override=outputFormat=csv")
resultfile = joinpath(getWorkDirectory(Kundur), resultfile_name)
df = DataFrame(CSV.File(resultfile));
(A, B, C, D) = linearize(Kundur)

# Function to compute natural frequency and damping ratio
function compute_freq_damping(eigenvalues)
    results = []
    
    for λ in eigenvalues
        # Extract real and imaginary parts
        α = real(λ)
        β = imag(λ)
        
        # Compute natural frequency
        f_n = sqrt(α^2 + β^2) / (2 * π)
        
        # Compute damping ratio
        ζ = -α / sqrt(α^2 + β^2)
        
        # Append the result (frequency, damping)
        push!(results, (f_n, ζ))
    end
    
    return results
end

# Compute frequency and damping ratio
freq_damping = compute_freq_damping(eigvals(A))

# Display results
for (f_n, ζ) in freq_damping
    println("Natural Frequency: $f_n Hz, Damping Ratio: $ζ")
end




LOG_STDOUT        | info    | Using sparse solver for linear system 0,
|                 | |       | because density of 0.066 remains under threshold of 0.200.
LOG_STDOUT        | info    | The maximum density for using sparse solvers can be specified
|                 | |       | using the runtime flag '<-lssMaxDensity=value>'.
LOG_STDOUT        | info    | Using sparse solver kinsol for nonlinear system 0 (638),
|                 | |       | because density of 0.05 remains under threshold of 0.10.
LOG_STDOUT        | info    | Using sparse solver kinsol for nonlinear system 1 (1451),
|                 | |       | because density of 0.05 remains under threshold of 0.10.
LOG_STDOUT        | info    | The maximum density for using sparse solvers can be specified
|                 | |       | using the runtime flag '<-nlssMaxDensity=value>'.
LOG_STDOUT        | info    | Linearization will be performed at point of time: 20.000000
LOG_SUCCESS       | info    | The initialization finished 

Internal error: stack overflow in type inference of linearized_model().
This might be caused by recursion over very long tuples or argument lists.
